In [50]:
import re
import json
import pyautogui
from time import sleep
from selenium import webdriver
import random

In [51]:
class FollowerCollector:

    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.dic_id_follower = {}
        self.window_width = 0
        self.window_height = 0
        self.driver = webdriver.Chrome(executable_path=r"C:\Users\care-lab\chromedriver.exe")

    def get_num_follower(self):
        html_source = self.driver.page_source
        pattern = '"edge_followed_by":{"count":(.*?)}'
        results = re.findall(pattern, html_source, re.S)
        return results[0]

    def scroll(self):
        pyautogui.scroll(-5*self.window_height)

    def do_infinite_scroll(self):
        num_follower = self.get_num_follower()
        if int(num_follower) == 0 or int(num_follower) > 500000:
            return 0

        followers_list01 = self.get_follower_list()
        while True:
            self.window_width, self.window_height = pyautogui.size()
            pyautogui.moveTo(self.window_width/2, self.window_height/2, duration=0)

            for i in range(5):
                scroll()
                sleep(1)

            followers_list02 = self.get_follower_list()
            print("-----Now getting followers({}/{})-----".format(len(followers_list01), num_follower))

            if len(followers_list01) != len(followers_list02):
                followers_list01 = followers_list02
            else:
                break

    def get_follower_list(self):
        html_source = self.driver.page_source
        pattern = '<a class="FPmhX notranslate _0imsa " title="(.*?)" href="'
        results = re.findall(pattern, html_source, re.S)
        return results

    def login(self):
        self.driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
        sleep(random.randint(5,10))
        self.driver.find_element_by_name('username').send_keys(self.username)#ユーザー名を入力してください
        sleep(random.randint(5,10))
        self.driver.find_element_by_name('password').send_keys(self.password)#パスワードを入力してください
        sleep(random.randint(5,10))
        #ログインボタンを押す
        self.driver.find_element_by_class_name('L3NKy       ').click()
        sleep(random.randint(5,10))
        """
        self.driver.find_elements_by_tag_name('input')[0].send_keys(self.username)
        self.driver.find_elements_by_tag_name('input')[1].send_keys(self.password)    
        sleep(1)

        self.driver.find_elements_by_tag_name('button')[1].click()

        sleep(3)
        """
    def display_followers(self, target_username):
        url = "https://www.instagram.com/{}/followers".format(target_username)
        self.driver.get(url)
        self.driver.maximize_window()

        sleep(1)

        element_follower_button_xpath = '//*[@id="react-root"]/section/main/div/header/section/ul/li[2]/a'
        self.driver.find_element_by_xpath(element_follower_button_xpath).click()

    def get_followers(self):
        sleep(3)
        
        self.do_infinite_scroll()
        print("infinite_scroll")
        self.followers_list = self.get_follower_list()
        return self.followers_list

    def save_idlist2json_file(self, id_i):
        file_name = "follower/{}".format(id_i)
        fw = open(file_name,'w')
        json.dump(self.dic_id_follower[id_i], fw, indent=4)

    def main(self, id_list):
        self.login()

        for id_i in id_list:
            try:
                self.display_followers(target_username=id_i)
                print(1)
                followers_list = self.get_followers()
                print(2)
                self.dic_id_follower[id_i] = followers_list
                print(3)
                self.save_idlist2json_file(id_i)
                print(4)
            except:
                print(5)
                self.dic_id_follower[id_i] = []
                print(6)
        self.driver.quit()
        return self.dic_id_follower

## ログインまで成功

In [52]:
if __name__ == '__main__':
    fc = FollowerCollector(username="shokoshi2", password="evnLEX8P")
    id_list = ["id_1", "id_2"]
    fc.main(id_list)

5
6
1
5
6
